**Index**
1. Load llm
2. Chain
3. Simple Sequential Chain
4. Sequiential Chain
5. Agents
6. Memory

In [1]:
import os
from secret_key import openapi_key
os.environ["OPENAI_API_KEY"] = openapi_key

In [11]:
# Help to use openai llm
from langchain.llms import OpenAI

llm  = OpenAI(temperature = 0.8)
name = llm('who is Bhavesh ?')

In [12]:
# Help in creating a prompt which can be feed to model
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(input_variables = ['cuisine'],
    template = "I want to open a restaurant for {cuisine} food . Suggest a fancy name for it"
)

prompt_template_name.format(cuisine = 'Indian')

'I want to open a restaurant for Indian food . Suggest a fancy name for it'

In [13]:
# Langchain combine llm and prompt using chain object 
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt = prompt_template_name)

chain.run('Indian')

'\n\n"Spice Odyssey"'

In [5]:
# Sequential chain it used to pass the previous output of llm as input again to it 

from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(input_variables = ['cuisine'],
    template = "I want to open a restaurant for {cuisine} food . Suggest a fancy name for it"
)

name_chain = LLMChain(llm=llm, prompt = prompt_template_name)

prompt_template_items = PromptTemplate(input_variables=['restaurant_name'],
                                       template= """Suggest some menu items for {restaurant_name} for each name, return in format of dictionary""")

food_items_chain = LLMChain(llm=llm, prompt = prompt_template_items)

In [6]:
print(food_items_chain.run('Spice Palace'))



1. "Sizzling Chicken Tikka Masala"
- Chicken Tikka Masala: Marinated chicken cooked in a creamy tomato sauce with a blend of spices, served sizzling hot.
- Garlic Naan: Traditional Indian flatbread brushed with garlic butter and cooked in a tandoor oven.
- Vegetable Samosas: Crispy pastry filled with spiced potatoes and mixed vegetables.
- Mango Lassi: Refreshing yogurt-based drink with a hint of mango.

2. "Spicy Lamb Vindaloo"
- Lamb Vindaloo: Tender chunks of lamb cooked in a spicy and tangy tomato-based sauce with a mix of traditional Indian spices.
- Basmati Rice: Fragrant long-grain rice cooked with aromatic spices.
- Onion Bhaji: Fried fritters made with sliced onions, gram flour, and spices.
- Spiced Chai: Traditional Indian black tea infused with a blend of warm spices.

3. "Paneer Tikka Masala"
- Paneer Tikka Masala: Cubes of paneer (Indian cottage cheese) cooked in a rich and creamy tomato sauce with a masala spice blend.
- Garlic and Coriander Naan: Naan bread flavored wi

In [8]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains = [name_chain,food_items_chain])

response = chain.run('Indian')
print(response)



1. "Royal Butter Chicken" - Tender chicken cooked in a rich and creamy tomato butter sauce, served with basmati rice and naan bread.
2. "Saffron Lamb Korma" - Slow-cooked lamb in a fragrant saffron-infused curry sauce, served with naan bread and cumin rice.
3. "Vegetable Biryani" - Aromatic basmati rice cooked with mixed vegetables and spices, topped with fried onions and served with raita.
4. "Tandoori Shrimp Masala" - Marinated shrimp cooked in a clay oven and finished in a creamy masala sauce, served with basmati rice and garlic naan bread.
5. "Paneer Makhani" - Cubes of paneer cheese cooked in a creamy tomato and cashew nut sauce, served with naan bread and cumin rice.
6. "Chicken Tikka Masala" - Grilled chicken pieces cooked in a spicy tomato and onion gravy, served with basmati rice and garlic butter naan bread.
7. "Dal Maharaja" - Slow-cooked lentils with spices, served with basmati rice, naan bread, and a side of pickles and chut


In [13]:
# Sequential chain it used to pass the previous output of llm as input again to it 

from langchain.prompts import PromptTemplate
llm  = OpenAI(temperature = 0.8)
prompt_template_name = PromptTemplate(input_variables = ['cuisine'],
    template = "I want to open a restaurant for {cuisine} food . Suggest a fancy name for it"
)

name_chain = LLMChain(llm=llm, prompt = prompt_template_name, output_key = "restaurant_name")

llm  = OpenAI(temperature = 0.8)
prompt_template_items = PromptTemplate(input_variables=['restaurant_name'],
                                       template= """Suggest some menu items for {restaurant_name} for each name.""")

food_items_chain = LLMChain(llm=llm, prompt = prompt_template_items,output_key = "menu_items")

In [14]:
# Sequntial chain help use to see intermediate output as well

from langchain.chains import SequentialChain

chain = SequentialChain(chains = [name_chain,food_items_chain],
                        input_variables = ['cuisine'],
                        output_variables = ["restaurant_name","menu_items"])


response = chain({'cuisine':'Indian'})
print(response)

c:\NLP\LLM\Langchain\langchain_env\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


{'cuisine': 'Indian', 'restaurant_name': '\n\n"Maharaja\'s Palace: Fine Indian Cuisine"', 'menu_items': '\n\n1. Chicken Tikka Masala\n2. Lamb Vindaloo\n3. Vegetable Biryani\n4. Tandoori Salmon\n5. Palak Paneer\n6. Butter Chicken\n7. Dal Makhani\n8. Aloo Gobi\n9. Rogan Josh (lamb curry)\n10. Baingan Bharta (roasted eggplant dish)\n11. Samosas (potato and pea filled pastries)\n12. Naan bread with garlic and cilantro\n13. Mango Lassi (yogurt-based drink)\n14. Gulab Jamun (syrup-soaked dumplings)\n15. Kheer (rice pudding)'}


**AGENTS**

Separate plugins which can be used by llm to anser the questions for which it don't have knowledge for  


In [8]:
from langchain.agents import  AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

#Loadding tools which we want our llm to use
tools = load_tools(['wikipedia','llm-math'],llm=llm)

#intiallizing agents with tools, llm and method it should use
agent = initialize_agent(tools,
                 llm,
                 agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION)

agent.run("who win most medal in paris olmpyic? What is his right now in 2024?")

'The United States won the most medals at the 2024 Summer Olympics, with 40 gold medals and 126 medals in total. The most decorated athlete at the 2024 Olympics was Sir Jason Kenny from Great Britain, who won seven gold medals and nine overall in track cycling.'

**Memory**
Remembering of converstation is required in case of customer support etc. Chain do not have memory hence we need toapply it separately

In [14]:
dir(chain)

['Config',
 'InputType',
 'OutputType',
 '__abstractmethods__',
 '__annotations__',
 '__call__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__orig_bases__',
 '__parameters__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__ror__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_updat

In [15]:
type(chain.memory)

NoneType

In [28]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt = prompt_template_name,memory = memory)

chain.run('Indian')

'\n\n"Taj Mahal Spice Co."'

In [17]:
chain.run('American')

'\n\nThe Star-Spangled Bistro '

In [18]:
chain.memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Indian'), AIMessage(content='\n\n"Indigo Spice" '), HumanMessage(content='American'), AIMessage(content='\n\nThe Star-Spangled Bistro ')]))

In [19]:
print(chain.memory.buffer)

Human: Indian
AI: 

"Indigo Spice" 
Human: American
AI: 

The Star-Spangled Bistro 


In [29]:
# the conversation grow endlessly hence need to but some trimming value or upper limit. we will use Conversation chain to do this

from langchain.chains import ConversationChain
convo  = ConversationChain(llm = OpenAI(temperature=0.7),memory=memory)
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [30]:
convo.run('who won last cricket world cup?')

' The last Cricket World Cup was won by England in 2019.'

In [31]:
convo.run('what is 5+6')

' 11 '

In [32]:
convo.run('who is captian of winning team?')

' The captain of the winning team, England, is Eoin Morgan.'

In [34]:
# setting up threshold
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

convo  = ConversationChain(llm = OpenAI(temperature=0.7),memory=memory)
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [35]:
convo.run('who won last cricket world cup?')

" The last cricket world cup was won by England in 2019. It was a historic win for England as it was their first time winning the world cup. The final match was played between England and New Zealand, and it was an intense and thrilling game. The match ended in a tie, but England was declared the winner based on the boundary count rule. It was a controversial finish, but England's team and fans were ecstatic with the win."

In [36]:
convo.run('what is 5+6')

' 5 + 6 is equal to 11.'

In [37]:
convo.run('who is captian of winning team?')

' I do not have enough information to answer that question accurately. However, if you are referring to a specific team, I may be able to provide you with the name of the captain. Can you give me more context?'